In [8]:
%idle_timeout 60
%timeout 30
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import pyspark.sql.functions as f
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 32b6ab4c-733b-46d8-bcc8-e4b14f55d12c.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 60 minutes.
idle_timeout has been set to 60 minutes.


You are already connected to a glueetl session 32b6ab4c-733b-46d8-bcc8-e4b14f55d12c.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current timeout is 30 minutes.
timeout has been set to 30 minutes.


You are already connected to a glueetl session 32b6ab4c-733b-46d8-bcc8-e4b14f55d12c.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 5.0


You are already connected to a glueetl session 32b6ab4c-733b-46d8-bcc8-e4b14f55d12c.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 32b6ab4c-733b-46d8-bcc8-e4b14f55d12c.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



In [9]:
raw_bucket = 'cloud9-batch-raw'

In [10]:
df = spark.read \
          .option("header", "true") \
          .option("encoding", "utf-8") \
          .csv(f's3://{raw_bucket}/fire_station/')

df.printSchema()
df.show(5, False)

root
 |-- 상위 본부명: string (nullable = true)
 |-- 119안전센터명: string (nullable = true)
 |-- 주소: string (nullable = true)
 |-- 전화번호: string (nullable = true)
 |-- FAX: string (nullable = true)
 |-- X좌표: string (nullable = true)
 |-- Y좌표: string (nullable = true)
 |-- 연도: string (nullable = true)
 |-- 근무인원수: string (nullable = true)

+----------------+-------------+---------------------------------------+------------+------------+-----------+-----------+----+----------+
|상위 본부명     |119안전센터명|주소                                   |전화번호    |FAX         |X좌표      |Y좌표      |연도|근무인원수|
+----------------+-------------+---------------------------------------+------------+------------+-----------+-----------+----+----------+
|서울소방재난본부|강남소방서   |서울특별시 강남구 테헤란로 629 (삼성동)|02-6981-7408|02-556-2119 |37.51027919|127.0668302|2017|325       |
|서울소방재난본부|강동소방서   |서울특별시 강동구 성내로 39 (성내동)   |02-6981-7600|02-6981-7717|37.52945788|127.1253721|2017|266       |
|서울소방재난본부|강북소방서   |서울특별시 강북구 한천로 911 (번동)    |02-6946-010

In [11]:
new_cols = ["fire_station_name", "fire_station_center_name", "address", "phone", "fax", "x-axis", "y-axis", "year", "employee_count"]

for i in range (0, len(df.columns)):
    df = df.withColumnRenamed(df.columns[i], new_cols[i])
    
# 결과 확인
df.printSchema()
df.show(5, False)

root
 |-- fire_station_name: string (nullable = true)
 |-- fire_station_center_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- fax: string (nullable = true)
 |-- x-axis: string (nullable = true)
 |-- y-axis: string (nullable = true)
 |-- year: string (nullable = true)
 |-- employee_count: string (nullable = true)

+-----------------+------------------------+---------------------------------------+------------+------------+-----------+-----------+----+--------------+
|fire_station_name|fire_station_center_name|address                                |phone       |fax         |x-axis     |y-axis     |year|employee_count|
+-----------------+------------------------+---------------------------------------+------------+------------+-----------+-----------+----+--------------+
|서울소방재난본부 |강남소방서              |서울특별시 강남구 테헤란로 629 (삼성동)|02-6981-7408|02-556-2119 |37.51027919|127.0668302|2017|325           |
|서울소방재난본부 |강동소방서            

In [12]:
df.write \
  .mode("overwrite") \
  .option("header", "true") \
  .partitionBy("year") \
  .csv(f's3a://{raw_bucket}/fire_station_raw_partitioned/')

spark.sql(
    """
    CREATE EXTERNAL TABLE IF NOT EXISTS cloud9_raw.rw_fire_station (
        `fire_station_name` string, 
        `fire_station_center_name` string, 
        `address` string, 
        `phone` string, 
        `fax` string, 
        `x-axis` double, 
        `y-axis` double, 
        `employee_count` bigint
    )
    PARTITIONED BY ( 
        `year` bigint
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY ',' 
    STORED AS INPUTFORMAT 
        'org.apache.hadoop.mapred.TextInputFormat' 
    OUTPUTFORMAT 
        'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION
        's3://cloud9-batch-raw/fire_station_raw_partitioned'
    TBLPROPERTIES (
        'classification'='csv'
    )
    """
)

spark.sql("MSCK REPAIR TABLE cloud9_raw.rw_fire_station")

DataFrame[]
